In [22]:
import os
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog

# Function to fix some bugs
def process_dataframe(df):
    # Check if values from first column to column 1289 in the last row are NaN
    if df.iloc[-1, :1289].isna().all():
        # Assign the values contained in columns 1290 and 1291 of the last row to variables
        col_1290_value = df.iloc[-1, 1289]
        col_1291_value = df.iloc[-1, 1290]
        
        # Delete the last row
        df = df.iloc[:-1, :]
        
        # Assign the values to the new last row
        df.iloc[-1, 1289] = col_1290_value
        df.iloc[-1, 1290] = col_1291_value
    
    return df

def append_columns_to_riassunto(riassunto, columns, values):
    for col, val in zip(columns, values):
        riassunto[col] = val
    return riassunto

def count_files(code_list):
    code_count = {}
    for code in code_list:
        if code in code_count:
            code_count[code] += 1
        else:
            code_count[code] = 1
    return code_count

def count_items(item_list):
    return len(item_list)

# Function to convert column to datetime format from row 20 to the last
def convert_column_to_datetime(df, column_index, start_row, date_format):
    for i in range(start_row, len(df)):
        try:
            df.iloc[i, column_index] = pd.to_datetime(df.iloc[i, column_index], format=date_format)
        except ValueError:
            # Skip rows that cannot be converted
            continue

# Loop through the list of dataframes and apply the function
def apply_conversion_to_dataframes(dataframes, column_index, start_row, date_format):
    for i, df in enumerate(dataframes):
        convert_column_to_datetime(df, column_index, start_row, date_format)
#_______________________________________________________________________________________________________________


# Directory containing the txt files
directory = r'V:\TEST CHILLER\TXT'

# List to store dataframes and skipped files
dataframes = []
skipped_files = []

# Loop through all files in the directory
for i, filename in enumerate(os.listdir(directory)):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        try:
            # First method to read the file
            df = pd.read_csv(file_path, delimiter="\t", header=None)
            df = df.drop(17).reset_index(drop=True)
        except Exception as e:
            try:
                # Second method to read the file if the first fails
                total_columns = 321
                df1 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', nrows=17)
                df2 = pd.read_csv(file_path, header=None, engine='python', encoding='latin', sep='\t', skiprows=18)
                df = pd.concat([df1, df2], axis=0).reset_index(drop=True)
            except Exception as e:
                # If both methods fail, skip the file and add to skipped_files list
                skipped_files.append(filename)
                continue
        
        # Add 'File name' and 'Last modification' columns at row 20
        df.loc[19, 'File name'] = filename
        df.loc[19, 'Last modification'] = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%d/%m/%Y %H:%M:%S')
        
        # Append the dataframe to the list
        dataframes.append(df)
        
        # Assign the dataframe to a variable name sequentially
        globals()[f'df_{i+1}'] = df

apply_conversion_to_dataframes(dataframes, 1, 20, '%d/%m/%Y %H:%M:%S')


In [23]:
# Function to check if all DataFrames have the same number of columns
def check_columns_consistency(dataframes):
    if not dataframes:
        return True  # If the list is empty, return True
    
    # Get the number of columns in the first DataFrame
    num_columns = dataframes[0].shape[1]
    
    # Check if all DataFrames have the same number of columns
    for df in dataframes:
        if df.shape[1] != num_columns:
            return False
    
    return True

consistent_columns = check_columns_consistency(dataframes)
consistent_columns

False

In [24]:
# Function to create a DataFrame with the name of the DataFrame and the number of columns
def create_summary_dataframe(dataframes):
    summary_data = []
    for i, df in enumerate(dataframes):
        df_name = f"df{i+1}"
        summary_data.append({"DataFrame Name": df_name, "Number of Columns": df.shape[1]})
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Create the summary DataFrame
summary_df = create_summary_dataframe(dataframes)
# Export the DataFrame to a CSV file
#summary_df.to_csv("summary_output2.csv", index=False)



In [25]:
# Function to split the list of DataFrames into two sublists based on the number of columns
import pandas as pd

def split_dataframes(dataframes):
    dataframes_323 = []
    dataframes_318 = []
    dataframes_323_names = []
    dataframes_318_names = []

    for df in dataframes:
        if df.shape[1] == 323:
            dataframes_323.append(df)
            dataframes_323_names.append(df.name)
        elif df.shape[1] == 318:
            dataframes_318.append(df)
            dataframes_318_names.append(df.name)

    return dataframes_323, dataframes_318, dataframes_323_names, dataframes_318_names

# Assign names to the sample dataframes
for i, df in enumerate(dataframes):
    df.name = f'df{i+1}'

# Split the list of DataFrames
dataframes_323, dataframes_318, dataframes_323_names, dataframes_318_names = split_dataframes(dataframes)

# Display the lists of DataFrame names
print("DataFrames with 323 columns:", dataframes_323_names)
print("DataFrames with 318 columns:", dataframes_318_names)


DataFrames with 323 columns: ['df1', 'df2', 'df3', 'df4', 'df5', 'df6', 'df7', 'df8', 'df12', 'df13', 'df14', 'df28', 'df32', 'df33', 'df34', 'df35', 'df36', 'df37', 'df38', 'df48', 'df49', 'df50', 'df51', 'df52', 'df55', 'df56', 'df57', 'df58', 'df59', 'df60', 'df64', 'df65', 'df66', 'df67', 'df68', 'df69', 'df70', 'df71', 'df72', 'df73', 'df74', 'df75', 'df76', 'df77', 'df78', 'df80', 'df81', 'df82', 'df83', 'df84', 'df85', 'df86', 'df87', 'df88', 'df89', 'df90', 'df91', 'df93', 'df94', 'df95', 'df96', 'df97', 'df99', 'df100', 'df101', 'df102', 'df103', 'df104', 'df105', 'df106', 'df107', 'df108', 'df109', 'df110', 'df111', 'df113', 'df114', 'df115', 'df116', 'df117', 'df118', 'df119', 'df120', 'df121', 'df122', 'df123', 'df124', 'df125', 'df126', 'df127', 'df128', 'df129', 'df130', 'df131', 'df132', 'df133', 'df134', 'df135', 'df144', 'df145', 'df146', 'df147', 'df148', 'df149', 'df150', 'df151', 'df152', 'df153', 'df154', 'df155', 'df156', 'df157', 'df158', 'df159', 'df160', 'df161

In [26]:
# Consider the first dataframe as the baseline
baseline = dataframes_318[0].iloc[17]

# Initialize the exceptions list
exceptions_318 = []

# Iterate over the dataframes starting from the second one
for df in dataframes_318[1:]:
    # Compare the 17th row with the baseline
    if not df.iloc[17].equals(baseline):
        # If there is a difference, add the dataframe to exceptions list
        exceptions_318.append(df)

# Remove the exceptions from the original list
dataframes_318 = [df for df in dataframes_318 if not any(df.equals(exc) for exc in exceptions_318)]

# Print the results
print(len(dataframes_318))
print(len(exceptions_318))

1
43


In [27]:
exceptions_318[0].head(20)

,0,1,2,3,4,5,6,7,8,9,...,308,309,310,311,312,313,314,315,File name,Last modification
0,STAZIONE DI COLLAUDO,R110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OPERATORE,1 CARLINI MIRCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CODICE MACCHINA,TAL57NNSBXIS2U0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MATRICOLA MACCHINA,00312502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NUMERO ODL,602680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,DATA/ORA COLLAUDO,20/03/2023 09:16:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FLUIDO,ACQUA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GAS REFRIGERANTE,R134a_PA.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,REVISIONE,02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LINEA DEL FLUIDO,BASSA PORTATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
column_names  = exceptions_318[0].iloc[50].tolist()
column_names[316] = 'File name'
column_names[317] = 'Last modification'
header = pd.DataFrame(columns=column_names)

# Initialize the lists
avg_columns = []
min_columns = []
max_columns = []

for col in header.columns[1:-2]:
    avg_columns.append(f"{col}_std")
    min_columns.append(f"{col}_min")
    max_columns.append(f"{col}_max")

# Add the new columns to the dataframe HEADER
for col in avg_columns + min_columns + max_columns:
    header[col] = None

#header.to_csv("header5.csv", index=False)


C:\Users\e2023898\AppData\Local\Temp\ipykernel_65428\4073072633.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  header[col] = None
C:\Users\e2023898\AppData\Local\Temp\ipykernel_65428\4073072633.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  header[col] = None
C:\Users\e2023898\AppData\Local\Temp\ipykernel_65428\4073072633.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

In [29]:
# Assuming all dataframes should have the same columns as the first dataframe
expected_columns = exceptions_318[0].columns

# Lists to hold dataframes with matching and different columns
matching_dataframes = []
different_dataframes = []

# Separate dataframes based on column names
for df in exceptions_318:
    if df.columns.equals(expected_columns):
        matching_dataframes.append(df)
    else:
        different_dataframes.append(df)

# Concatenate dataframes with matching columns
if matching_dataframes:
    final_dataframe = pd.concat(matching_dataframes, ignore_index=True)
    # Sort the dataframe by 'Last modification'
    final_dataframe.sort_values(by='Last modification', inplace=True)
else:
    final_dataframe = pd.DataFrame()

# Print the list of dataframes with different columns
print("Dataframes with different columns:")
for df in different_dataframes:
    print(df.columns)

Dataframes with different columns:


In [44]:
# Check if the entry at row 20 in the 'File name' column is of string type
all_strings = True
for df in exceptions_318:
    try:
        entry = df.at[19, 'File name']
        if not isinstance(entry, str):
            all_strings = False
            break
    except KeyError:
        all_strings = False
        break

print(f"All dataframes have a string entry at row 20 in the 'File name' column: {all_strings}")

All dataframes have a string entry at row 20 in the 'File name' column: True


In [45]:
for df in exceptions_318:
    value_to_copy = df.at[19, "File name"]
    df.loc[19:, "File name"] = value_to_copy


In [56]:
import numpy as np

riassunto_dataframes = []
column_names  = exceptions_318[0].iloc[16].tolist()
column_names[316] = 'File name'
column_names[317] = 'Last modification'
baseline_df = exceptions_318[0].iloc[:16]
baseline_df = baseline_df.loc[:, baseline_df.notna().any(axis=0)]
#baseline_df = baseline_df.drop(14)
baseline_first_column = baseline_df.iloc[:, 0].tolist()
dataframe_318_columns_names =exceptions_318[0].iloc[50].tolist()


def replace_line_with_list(df, line_index, values_list):
    # Replace the values of the specified line with the values from the list
    df.iloc[line_index] = values_list
    return df

#column_names = header.columns.tolist()

# Process each dataframe and create riassunto dataframes
for i, df in enumerate(exceptions_318):
    # Split the dataframe
    test_id = df.iloc[:17]
    test_id = test_id.loc[:, test_id.notna().any(axis=0)]
    index_row = df.iloc[16]
    remaining_df = df.iloc[16:]
    #test_id = test_id.drop(14)
    #test_data = test_id.transpose()
    #test_data.columns = test_data.iloc[0]
    #test_data = test_data[1:]
    #test_data_values = test_data.iloc[0]
    df = replace_line_with_list(df, 16, dataframe_318_columns_names)
    df.columns.values[-2:] = ['File name', 'Last modification']

    aligned_test_id = pd.DataFrame(columns=test_id.columns)
    for value in baseline_first_column:
            if value in test_id.iloc[:, 0].values:
                row_index = test_id[test_id.iloc[:, 0] == value].index[0]
                aligned_test_id = pd.concat([aligned_test_id, test_id.loc[[row_index]]])
            else:
                new_row = pd.Series([value] + ["Value not declared"] * (test_id.shape[1] - 1), index=test_id.columns)
                aligned_test_id = pd.concat([aligned_test_id, new_row.to_frame().T], ignore_index=True)

    # Append the remaining rows
    aligned_test_id.reset_index(drop=True, inplace=True)

    # Initialize variables
    datasets = {}
    current_dataset = []
    dataset_counter = 1
    note_counter = 0

    # Process the remaining rows
    for index, row in remaining_df.iterrows():
        if row[1] == "Note":
            note_counter += 1
            if note_counter > 1:
                datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)
                dataset_counter += 1
                current_dataset = []
        current_dataset.append(row)

    # Append the last dataset if exists
    if current_dataset:
        datasets[f"dataset_{dataset_counter}"] = pd.DataFrame(current_dataset)

    # Create a dataframe for each dataset and show the names of the dataframes created

    #first cell (first row, first column) of the dataset
    for key, dataset in datasets.items():
        first_column_value = dataset.iloc[2, 0]
        dataset.iloc[0, 0] = first_column_value

    dataframe_names = []

    for key, dataset in datasets.items():
        globals()[key] = dataset
        dataframe_names.append(key)

    # Apply the operations to each dataset
    for key, dataset in datasets.items():

        # Replace all commas with periods in columns 2 to the end
        dataset.iloc[:, 1:] = dataset.iloc[:, 1:].replace(',', '.', regex=True)
        
        # Convert subsequent columns to float, coercing errors
        for col in dataset.columns[2:-2]:
            dataset[col] = pd.to_numeric(dataset[col], errors='coerce')
            dataset.iloc[2:, dataset.columns.get_loc(col)] = pd.to_numeric(dataset.iloc[2:, dataset.columns.get_loc(col)], errors='coerce')
        
        # Update the dataset in the dictionary
        datasets[key] = dataset

    # Create the dataframe for storing general data
    riassunto = pd.DataFrame(columns=column_names)
    new_columns = []
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=new_columns)], axis=1)

    for key, dataset in datasets.items():
        if len(dataset) < 4:
            time_delta = "Cannot be calculated"
        else:
            # Ensure the values are Timestamps
            start_row_index = 3
            start_time = dataset.iloc[start_row_index, 1]
            
            # Check if start_time is valid
            while pd.isna(start_time) or isinstance(start_time, str):
                start_row_index += 1
                if start_row_index >= len(dataset):  # Ensure we don't go out of bounds
                    start_time = None
                    break
                start_time = dataset.iloc[start_row_index, 1]
            
            end_row_index = len(dataset) - 1
            end_time = dataset.iloc[end_row_index, 1]
            
            # Check if end_time is valid
            while pd.isna(end_time) or isinstance(end_time, str):
                end_row_index -= 1
                if end_row_index < start_row_index:  # Ensure we don't go out of bounds
                    end_time = None
                    break
                end_time = dataset.iloc[end_row_index, 1]
            
            if start_time is None or end_time is None:
                time_delta = "Cannot be calculated"
            else:
                time_delta = end_time - start_time
        
        row_index = dataframe_names.index(key)
        riassunto.at[row_index, riassunto.columns[1]] = time_delta

    # Assign values from datasets to riassunto sequentially in column 1##########################################################################################
    row_counter = 0
    for key, dataset in datasets.items():
        if len(dataset) < 2:
            value_to_assign = "Not enough data"
            riassunto.at[row_index, 'Note'] = value_to_assign
        else:
            value_to_assign = dataset.iloc[1, 0]  # Value from column 1 row 2
            row_index = dataframe_names.index(key)
            riassunto.at[row_index, 'Note'] = value_to_assign
            row_counter += 1

    # Iterate over each dataset
    for key, dataset in datasets.items():

        # Calculate the average values for each column starting from the third column
        averages = dataset.iloc[2:, 2:-2].mean()
        
        # Assign the average values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        
        for col_index, avg_value in enumerate(averages, start=2):
            riassunto.iloc[row_index, col_index] = avg_value

    # Create new columns for minimum and maximum values
    avg_columns = []
    min_columns = []
    max_columns = []

    for col in riassunto.columns[2:-2]:
        avg_columns.append(f"{col}_std")
        min_columns.append(f"{col}_min")
        max_columns.append(f"{col}_max")
        

    # Append new columns to riassunto dataframe
    riassunto = pd.concat([riassunto, pd.DataFrame(columns=avg_columns + min_columns + max_columns)], axis=1)

    # Iterate over each dataset
    for key, dataset in datasets.items():
        # Calculate the minimum and maximum values for each column starting from the third column
        deviations = dataset.iloc[2:, 2:-2].std()
        min_values = dataset.iloc[2:, 2:-2].min()
        max_values = dataset.iloc[2:, 2:-2].max()
        
        # Assign the minimum values to the corresponding row in riassunto
        row_index = dataframe_names.index(key)
        for col_index, deviations in enumerate(deviations, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = deviations
        
        for col_index, min_value in enumerate(min_values, start=2):
            riassunto.iloc[row_index, len(riassunto.columns)//3 + col_index] = min_value
        
        # Assign the maximum values to the corresponding row in riassunto
        for col_index, max_value in enumerate(max_values, start=2):
            riassunto.iloc[row_index, 2*len(riassunto.columns)//3 + col_index] = max_value

# Assign the maximum values to the corresponding row in riassunto
    # Check if 'File name' and 'Last modification' columns exist before assigning
    if 'File name' in df.columns and 'Last modification' in df.columns:
        riassunto['File name'] = df['File name'].iloc[19]
        riassunto['Last modification'] = df['Last modification'].iloc[19]
    else:
        riassunto['File name'] = 'Data not found'
        riassunto['Last modification'] = 'Data not found'
    riassunto.reset_index(drop=True, inplace=True)
    updated_riassunto = append_columns_to_riassunto(riassunto, aligned_test_id.transpose().iloc[0], aligned_test_id.transpose()[1:].iloc[0]) 

# Assin the riassunto dataframe to a variable name sequentially
    globals()[f'riassunto_{i+1}'] = updated_riassunto
    riassunto_dataframes.append(updated_riassunto)



c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\e2023898\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [48]:
# Check which of the dataframes in the list 'riassunto_dataframes' does not have the same number of columns
def find_inconsistent_dataframes(dataframes):
    num_columns = [df.shape[1] for df in dataframes]
    max_columns = max(num_columns)
    inconsistent_dfs = [i for i, n in enumerate(num_columns) if n != max_columns]
    return inconsistent_dfs

# Find the inconsistent dataframes in riassunto_dataframes
inconsistent_dataframes = find_inconsistent_dataframes(riassunto_dataframes)

if inconsistent_dataframes:
    print(f"The following dataframes do not have the same number of columns: {inconsistent_dataframes}")
else:
    print("All dataframes have the same number of columns.")

All dataframes have the same number of columns.


In [57]:
# Create the global dataframe with modified column names
global_df = pd.DataFrame(columns=[col for col in header])

# Add another column with the name "N.TEST"
global_df["N.TEST"] = None

In [58]:
# Use the first dataframe of the list riassunto_dataframes as baseline
baseline_columns = riassunto_dataframes[0].columns

# Assign the value of its column names to all other dataframes in the list
for i in range(1, len(riassunto_dataframes)):
    riassunto_dataframes[i].columns = baseline_columns

# Assign a sequential test_id for each dataframe in riassunto_dataframes
test_ids = [f"t{i+1:03d}" for i in range(len(riassunto_dataframes))]

# Concatenate the rows of all dataframes in a global_df
global_df = pd.DataFrame(columns=baseline_columns)
global_df["N.TEST"] = None

for i, df in enumerate(riassunto_dataframes):
    df["N.TEST"] = test_ids[i]
    global_df = pd.concat([global_df, df], ignore_index=True)



In [18]:
# Function to read the value and data type of a cell in a dataframe
def read_cell_value_and_type(df, row_index, col_index):
    cell_value = df.iloc[row_index, col_index]
    cell_type = type(cell_value)
    return cell_value, cell_type

# Read the value and data type of the cell at row index 1 and column index 2
cell_value, cell_type = read_cell_value_and_type(global_df, 1, 6)

# Print the value and data type of the cell
print(f"The value of the cell at row index 1 and column index 2 is: {cell_value}")
print(f"The data type of the cell is: {cell_type}")

The value of the cell at row index 1 and column index 2 is: Value not declared
The data type of the cell is: <class 'str'>


In [61]:
# Loop through the DataFrame rows and round float values to two decimal places
for index, row in global_df.iterrows():
    for col in global_df.columns:
        if isinstance(row[col], float):
            global_df.at[index, col] = round(row[col], 2)

# Function to convert decimal separator from . to , for display purposes
def convert_decimal_separator(df):
    df_str = df.copy()
    for col in df_str.select_dtypes(include=['float']):
        df_str[col] = df_str[col].map(lambda x: f"{x:.2f}".replace('.', ','))
    return df_str

# Convert the decimal separator for display
global_df_display = convert_decimal_separator(global_df)

# Export the DataFrame to CSV with comma as the decimal separator
global_df_display.to_csv('exception318_comma_02.csv', index=False, sep=';')

print("DataFrame has been exported to 'global_df_comma.csv' with comma as the decimal separator.")

DataFrame has been exported to 'global_df_comma.csv' with comma as the decimal separator.


NameError: name 'aligned_test_id' is not defined